In [ ]:
!pip install tfx
!pip install transformers
!pip install pillow

In [1]:
data_path = "data"
model_file = "modules/model.py"
model_fn = "modules.model.run_fn"

In [2]:
import tfx
tfx.__version__

'1.9.1'

In [3]:
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import ImportExampleGen
from tfx.components import Trainer
from tfx.components import Pusher
from tfx.components import StatisticsGen
from tfx.proto import example_gen_pb2

In [4]:
context = InteractiveContext()

In [5]:
input_config = example_gen_pb2.Input(
    splits=[
        example_gen_pb2.Input.Split(name="train", pattern="train/*.tfrecord"),
        example_gen_pb2.Input.Split(name="eval", pattern="eval/*.tfrecord"),
    ]
)
example_gen = ImportExampleGen(
    input_base=data_path, 
    input_config=input_config
)

In [6]:
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [7]:
!mkdir modules

In [8]:
%%writefile {model_file}

from typing import List, Dict, Tuple

import absl
import tensorflow as tf
from transformers import ViTFeatureExtractor, TFViTForImageClassification
from tfx.components.trainer.fn_args_utils import FnArgs

feature_extractor = ViTFeatureExtractor()

_TRAIN_LENGTH = 128
_EVAL_LENGTH = 128
_TRAIN_BATCH_SIZE = 8
_EVAL_BATCH_SIZE = 8

_IMAGE_KEY = "image"
_LABEL_KEY = "label"

CONCRETE_INPUT = "pixel_values"
SIZE = feature_extractor.size
INPUT_SHAPE = (SIZE, SIZE, 3)

def INFO(text: str):
    absl.logging.info(text)

def _normalize_img(
    img, mean=feature_extractor.image_mean, std=feature_extractor.image_std
):
    # Scale to the value range of [0, 1] first and then normalize.
    img = img / 255
    mean = tf.constant(mean)
    std = tf.constant(std)
    return (img - mean) / std


def _preprocess(string_input):
    decoded_input = tf.io.decode_base64(string_input)
    decoded = tf.io.decode_jpeg(decoded_input, channels=3)
    resized = tf.image.resize(decoded, size=(SIZE, SIZE))
    normalized = _normalize_img(resized)
    normalized = tf.transpose(
        normalized, (2, 0, 1)
    )  # Since HF models are channel-first.
    return normalized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def _preprocess_fn(string_input):
    decoded_images = tf.map_fn(
        _preprocess, string_input, dtype=tf.float32, back_prop=False
    )
    return {CONCRETE_INPUT: decoded_images}


def _model_exporter(model: tf.keras.Model):
    m_call = tf.function(model.call).get_concrete_function(
        tf.TensorSpec(
            shape=[None, 3, SIZE, SIZE], dtype=tf.float32, name=CONCRETE_INPUT
        )
    )

    @tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
    def serving_fn(string_input):
        labels = tf.constant(list(model.config.id2label.values()), dtype=tf.string)
        images = _preprocess_fn(string_input)

        predictions = m_call(**images)
        indices = tf.argmax(predictions.logits, axis=1)
        pred_source = tf.gather(params=labels, indices=indices)
        probs = tf.nn.softmax(predictions.logits, axis=1)
        pred_confidence = tf.reduce_max(probs, axis=1)
        return {"label": pred_source, "confidence": pred_confidence}

    return serving_fn

def _parse_tfr(proto):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string), 
        "label": tf.io.FixedLenFeature([], tf.string)
    }
    rec = tf.io.parse_single_example(proto, feature_description)

    image = tf.io.parse_tensor(rec["image"], tf.float32)
    label = tf.io.parse_tensor(rec["label"], tf.float32)

    return {"pixel_values": image, "labels": label}

def _input_fn(
    file_pattern: List[str],
    batch_size: int = 32,
    is_train: bool = False,
) -> tf.data.Dataset:
    INFO(f"Reading data from: {file_pattern}")

    dataset = tf.data.TFRecordDataset(
        tf.io.gfile.glob(file_pattern[0] + ".gz"),
        num_parallel_reads=tf.data.AUTOTUNE,
        compression_type="GZIP",
    ).map(_parse_tfr, num_parallel_calls=tf.data.AUTOTUNE)

    if is_train:
        dataset = dataset.shuffle(batch_size * 2)

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    dataset = dataset.map(_preprocess)
    return dataset

def run_fn(fn_args: FnArgs):
    # train_dataset = _input_fn(
    #     fn_args.train_files,
    #     is_train=True,
    #     batch_size=_TRAIN_BATCH_SIZE,
    # )

    # eval_dataset = _input_fn(
    #     fn_args.eval_files,
    #     is_train=False,
    #     batch_size=_EVAL_BATCH_SIZE,
    # )

    # num_labels = 35
    # model = _build_model(num_labels)
    model = TFViTForImageClassification.from_pretrained("google/vit-base-patch16-224")

    # model.fit(
    #     train_dataset,
    #     steps_per_epoch=_TRAIN_LENGTH // _TRAIN_BATCH_SIZE,
    #     validation_data=eval_dataset,
    #     validation_steps=_EVAL_LENGTH // _TRAIN_BATCH_SIZE,
    #     epochs=_EPOCHS,
    # )

    model.save(
        fn_args.serving_model_dir, save_format="tf", signatures=_model_exporter(model)
    )

Writing modules/model.py


In [9]:
trainer = Trainer(
    run_fn=model_fn,
    examples=example_gen.outputs["examples"],
)

In [10]:
context.run(trainer)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFViTForImageClassification.

All the layers of TFViTForImageClassification were initialized from the model checkpoint at google/vit-base-patch16-224.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Instructions for updating:
Use fn_output_signature instead


ExecutionResult(
    component_id: Trainer
    execution_id: 2
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))